### 验证LLM的代码静态分析能力
https://github.com/CoReBench/CoRe/tree/main 
重新实现其功能

steps:
- copy prompt
- 找代码例子来测试   

### CORE实现
1. 给一段代码
2. 问大模型某个数据流分析的结果是否正确：
```
**Question**: Is there information flow from `(s,18)` to `(oline,26)` in function `ope_line`? If so, provide a trace.
```
3. 输出结果为Json，就能自动对比label文件去测试正确性
```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["j", 5],
      "to":   ["score", 11],
      "type": "data"
    }
  ]
}
```
4. 多轮次提问，提升准确率

In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
system_prompt = "You are a helpful program analysis expert."
MODEL = "qwen3-coder-plus"

def llm_chat(question, prompt=system_prompt):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": question}
        ],
        stream=False
    )
    return response.choices[0].message.content

llm_chat("Who are you?")

'I am Qwen, a large-scale language model independently developed by the Tongyi Lab under Alibaba Group. I am capable of answering questions, creating text such as stories, official documents, emails, and scripts, performing logical reasoning, coding, and more. I can also express opinions and play games. I support multiple languages, including Chinese, English, German, French, Spanish, Portuguese, Italian, Dutch, Russian, Czech, Polish, Arabic, Persian, Hebrew, Turkish, Japanese, Korean, and many others.'

In [20]:
c_prompt_1 = """
[INSTRUCTIONS]

You are a program-analysis assistant. Please perform a **static** analysis of “information flow” on a given code snippet, treating each branch or loop condition as potentially taking any outcome, without using semantic or symbolic execution to prune paths.

## 1. Information Flow Definition

An **information flow** from variable `x` to variable `y`, written `x -> y`, exists whenever information stored in `x` is transferred to or used to derive information transferred to `y`. 

We denote each variable instance as `(var,lineNumber)`, meaning the variable `var` is **defined or updated** at `lineNumber`.


### Types of Direct Flows

- **Direct Explicit Flow**  
  A direct explicit flow occurs when the value written by one variable instance is directly used in computing the value of another, through operations like assignments.

  Formally, if `(varA, lineA)` writes a value and `(varB, lineB)` reads that value to compute its own, then there is a **direct explicit flow** from `(varA, lineA)` to `(varB, lineB)`, written as: `(varA,lineA) -> (varB,lineB)`. 


  **Example 1**:
    ```
    1  x = 5
    2  y = x + 1
    ```
    `(x,1) -> (y,2)` is a direct explicit flow.


  **Example 2**:
    ```
    1  x = 5
    2  if cond:
    3      y = x + 1
    ```
    `(x,1) -> (y,3)` is a direct explicit flow. 

    Under static analysis, we treat **each conditional or loop as potentially taking any outcome**, and we **do not perform semantic pruning**. Therefore, even if the statement at line 3 is conditionally executed, we still recognize `(x,1)` as used to compute `(y,3)`.
    


- **Direct Implicit Flow**

  A direct implicit flow from `(varA,lineA)` to `(varB, lineB)` occurs when `(varA,lineA)`’s value directly influences whether `lineB` executes. In other words, it corresponds to a **direct control dependence**, where the condition has **at least one branch** where `(varB, lineB)` must run and another where it may not. All conditional structures (e.g., if, while, for, switch, etc.) generate these flows.

  Formally, if `(varA,lineA)` is **directly read** in a condition at `lineC` and that condition **directly** decides whether `lineB` executes, then we say there is an **direct implicit flow** from `(varA,lineA)` to `(varB,lineB)` (or `(varA,lineA) -> (varB,lineB)`).

  **Example**:
  ```
  1  z = x + 5
  2  if z > 1:
  3      y = 10
  ```
  `(z,1) -> (z,2)* -> (y,3)` is a direct implicit flow through the condition at line 2. Line 2 is a conditional statement and the value of `(z,2)*` here is not redefined, therefore marked with "*".
  
  The asterisk * indicates that the variable is used for the conditional statement but not redefined at that line (e.g., used in a condition).



### Information Flow Between Variables

An **information flow** exists from a variable `(varA, lineA)` to another variable `(varB, lineB)` if there is a **transitive chain of direct flows** — explicit, implicit, or both — connecting them.

That is, there exists a sequence of variables:
`(varA, lineA) -> ... -> (varB, lineB)`



## 2. **Output Format**  

When asked "Is there information flow from `(varA,lineA)` to `(varB,lineB)`? If so, provide one feasible trace.", respond with a JSON object:

- If information flow exists:
```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["varOrExpr", lineNumber            /*, "use" optional */],
      "to":   ["varOrExpr", lineNumber            /*, "use" optional */],
      "type": "data" | "control"
    },
    // … additional edges
  ]
}
```
- If no information flow exists, omit the Trace field:
```json
{
  "InformationFlow": false
}
```

- Every trace edge must specify "from", "to" (each may include a third "use" element when the line is a conditional statement that only reads without redefining/updating the value), and "type".
- **We do not consider loop in the trace**.


A **trace** represents a **transitive chain of direct flows** arise from compositions of **direct** implicit or explicit flows, where each edge represents a **direct implicit flow** (through control dependence) or a **direct explicit flow** edge (through data dependence).


You only need to provide **one** valid trace if you conclude a information flow exists, even if multiple possible chains exist. 


## 3. **Intraprocedural Data Dependence**  
   All dependence analysis is performed within **a single function**. We do not track dependencies across function boundaries. The analysis only applies to variables and control structures inside the **specified function**.


## 4. **Example Code Snippet**:


### Example 1
```python
1  def example_func():
2      status = 0
3      flag = False
4      balance = 1000
5      balance += 500
6      if balance > 1000:
7          status = 1
8          flag = True
9      limit = status * 5000
10     transaction = limit * 0.2
11     return flag
```

#### Example Question 1.1:
Is there information flow from `(balance,4)` to `(transaction,10)`? If so, provide a trace.

**Analysis**:
- Line 10: `(transaction,10)` reads `limit`, so there is a direct explicit flow from `(limit,9)` to `(transaction,10)`. 
- Line 9: `(limit,9)` reads `status`, so there is a direct explicit flow from `(status,7)` (or `(status,2)`) to `(limit,9)`. Here we focus on `(status,7)`. 
- Line 7: `(status,7)` is updated only if line 6’s condition is true, so has a direct control dependence on line 6.
- Line 6: checks `balance > 1000`, reads from `(balance,5)`, so there is a direct implicit flow from `(balance,5)` to `(status,7)` through `balance` at line 6.
- Line 5: `(balance,5)` reads from `(balance,4)`, so there is a direct explicit flow from `(balance,4)` to `(balance,5)`.
- Line 4: `(balance,4)` initializes the variable balance.
  
Hence, A transitive of flow of direct implicit or explicit flows forms a information flow trace.


**Output**:
```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["balance", 4],
      "to":   ["balance", 5],
      "type": "data"
    },
    {
      "from": ["balance", 5],
      "to":   ["balance", 6, "use"],
      "type": "data"
    },
    {
      "from": ["balance", 6, "use"],
      "to":   ["status", 7],
      "type": "control"
    },
    {
      "from": ["status", 7],
      "to":   ["limit", 9],
      "type": "data"
    },
    {
      "from": ["limit", 9],
      "to":   ["transaction", 10],
      "type": "data"
    }
  ]
}
```

 
- The edge from line 6 to `(status,7)` is through control dependence and therefore with a type of "control". 
- Line 6 only reads the value of `balance` without defining/updating it. So it is marked with `"use"`. 
- You only need to provide **one** valid chain if you conclude a the information flow exists. 


#### Example Question 1.2

Is there information flow from  `(flag,3)` to `(limit,9)`? If so, provide a trace.

**Analysis**
- Line 9: `(limit,9)` reads `status`, so there is a direct explicit flow from either `(status,7)` (if line 6’s condiation evaluates to `true`) or on the original `(status,2)` (if not) to `(limit,9)`. 
- Since this is a static analysis, all possible paths are considered without analyzing actual reachability. This means line 6 is treated as being evaluated to both possibly true and possibly false.
  - If line 6 is true, `(limit,9)` depends on `(status,7)`. 
    - Line 7: `(status,7)` doesn't have any data dependence as it is assigned with a constant number. The execution of line 7 is control-dependent on the condition in line 6.
    - Line 6: the condition at line 6 directly reads from `(balance,5)`. 
    - Line 5: `(balance,5)` is updated based on the previous value of `balance` in line 4.
    - Line 4: `(balance,4)` is initialized with a constant number. Therefore it has no more data dependence. Nor is it guarded by any conditions.
  - If line 6 is not true, `(limit,9)` depends on `(status,2)`. 
    - Line 2: `(status,2)` is initialized with a constant number. Therefore it has no more data dependence. Nor is it guarded by any conditions.
- Therefore, among all the possible paths, there is no transitive (indirect) chain of implicit or explicit flow from `(flag,3)` to `(limit,9)`. The information flow doesn't exist.


**Output**:
```json
{
  "InformationFlow": false
}
```

#### Example Question 1.3

Is there information flow from  `(status,2)` to `(transaction,10)`? If so, provide a trace.

**Analysis**
- Line 10: `(transaction,10)` reads `limit`, so there is a direct explicit flow from `(limit,9)` to `(transaction,10)`.
- Line 9: `(limit,9)` reads `status`, o there is a direct explicit flow from either `(status,7)`, or the original `(status,2)`, to `(limit,9)`.
- Similar to the above example, line 6 is possibly evaluated to true or false.
- If line 6 evaluates to `true`, there is a direct explicit flow from `(status,7)` to `(limit,9)`, which directly data-dependent on `(status,2)`, forming a transitive explicit flow trace :`(status,2) -> (status,7) -> (limit,9) -> (transaction,10)`.
- If line 6 evaluates to `false`, `(limit,9)` has a direct data dependence on `(status,2)`, forming a transitive explicit flow trace :`(status,2) -> (limit,9) -> (transaction,10)`.
- A transitive of explicit flow forms a information flow trace. Only one valid trace is required for the output even if multiple exists.

**Output**:
```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["status", 2],
      "to":   ["limit", 9],
      "type": "data"
    },
    {
      "from": ["limit", 9],
      "to":   ["transaction", 10],
      "type": "data"
    }
  ]
}
```


### Example 2
```python
1.  val = 5
2.  size = 3
3.  arr = [0] * size
4.  i = 0
5.  j = 1
6.  total = 0
7.  if val > 2:
8.      arr[j % size] = j
9.  while i < size:
10.     arr[j] += 1
11.     score = arr[j+1]
12.     total += score * 2
13.     diff = total - score
14.     j = (j + 1) % size
15.     i += 1
16. last = arr[-1]
17. summary = diff + j
```

#### Example Question 2.1:
Is there information flow from  `(size,2)` to `(summary,17)`? If so, provide a trace.

**Analysis**:
- Line 17: `(summary,17)` reads `diff` and `j`. We focus on the path through `j`. `j` is updated from line 14 and line 5. 
- Static analysis assumption: Every branch or loop condition may succeed or fail; we do not prune unreachable paths. When line 9 evaluates to `true`, line 14 will be executed. 
- Line 14: `(j,14)` is updated inside the `while i < size` loop, so its update is control-dependent on the loop condition at line 9.
- Line 9: `while i < size` reads `size` (a pure read), so has a direct explicit flow from `(size,2)`.
- Line 2: `size` is initialized with a constant number. It is not guarded with any conditions. 
- Therefore, there is an transitive trace of direct implicit/excplit flows from `(size,2)` to `(summary,17)`.

Only one valid trace is required for the output. 

**Output**:
```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["size", 2],
      "to":   ["size", 9, "use"],
      "type": "data"
    },
    {
      "from": ["size", 9, "use"],
      "to":   ["j", 14],
      "type": "control"
    },
    {
      "from": ["j", 14],
      "to":   ["summary", 17],
      "type": "data"
    }
  ]
}
```


#### Example Question 2.2:
Is there information flow from  `(j,5)` to `(score,11)`? If so, provide a trace. 

**Analysis**:
- Line 11: `(score,11)` reads `arr[j+1]`. There is a direct data dependence on both `arr` and `j`. That `j` may come from either `(j,5)` (initial) or subsequent updates at `(j,14)`.
- Static analysis assumption: Every branch or loop condition may succeed or fail; we do not prune unreachable paths. Therefore, `j` could be read from either line 5 (e.g., when it is the first iteration of the while loop) or line 14 (e.g., when it is the not the first iteration of the while loop).
- If line 11 reads `j` from line 5, it forms a tansitive explicit flow trace: `(j,5) -> (score,11)`.
- If line 11 reads `j` from line 14, it forms a tansitive explicit flow trace: `(j,5) -> (j,14) -> (score,11)`.
- A transitive of explicit flow forms a information flow trace. Only one valid trace is required for the output even if multiple exists.


**Output**:
```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["j", 5],
      "to":   ["score", 11],
      "type": "data"
    }
  ]
}
```



#### Example Question 2.3:
Is there information flow from  `(j,1)` to `(i,15)`? If so, provide a trace.

**Analysis**: 
- Line 15: `(i,15)` updates `i` based on its previous value. So there is a direct explicit flow from either `(i,4)` (initialization) or itself (loop-carried) to `(i,15)`. But we do not consider loop-carried dependencies across iterations in this analysis. So we focus on `(i,4)`.
- The execution of line 15 is conditional on the while loop at line 9.
- Line 9: `while i < size` depends on `i` and `size`. It directly reads from `(i,4)` (or possibly `(i,15)`) and `(size,2)`.
- Both `(i,4)` and `(size,2)` are initialized with constants and are not themselves control-dependent on any earlier values or branches.
- There is no use of `(j,1)` in the computation of `(i,15)`, nor does `(j,1)` influence whether line 15 executes.
- Therefore, there is no transitive flow, explicit or implicit, from `(j,1)` to `(i,15)`.


**Output**: 
```json
{
  "InformationFlow": false
}
```


#### Example Question 2.4:
Is there information flow from  `(val,1)` to `(last,16)`? If so, provide a trace.

**Analysis**:
- Line 16: `(last,16)` reads `arr[-1]`. That element may have been modified by the assignment in line 8 (or by the loop updates in line 10). Under static analysis, multiple update sites may contribute. We conservatively pick one valid trace (line 8).
- Line 8: `arr[j % size] = j` executes only if `val > 2`, so `(arr,8)` has a direct control dependence on `(val,1)` via the `if` at line 7.
- Therefore, there exists a trace from `(val,1)` to `(last,16)`, and the edge from line 7 to `(arr,8)` is through implicit and explicit flows. 

**Output**:
```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["val", 1],
      "to":   ["val", 7, "use"],
      "type": "data"
    },
    {
      "from": ["val", 7, "use"],
      "to":   ["arr", 8],
      "type": "control"
    },
    {
      "from": ["arr", 8],
      "to":   ["last", 16],
      "type": "data"
    }
  ]
}
```

---

[YOUR TURN]

Below is **your target snippet**. 

"""

In [23]:
# 当源码中转义字符'\n'被解释时，代码行号分配错误
def c_with_li_num(code):
	msg = "```c\n"
	for idx, li in enumerate(code.strip().split("\n")):
		msg += "{} {}\n".format(idx + 1, li)
	return msg + "```"

# 不解释转义字符
c_code_snipet_1 = r"""
#include <stdio.h>          
#include <stdbool.h>
const char NUL = '\0';
bool
ope_line()
{
	char oline[101];
	const char letter[6][5] = {"abcde", "fghij", "klmno", "pqrst", "uvwxy", "z.?! "};
	int c = getc(stdin);
	if (c == EOF)
		return false;
	if (c == '\n')
		return true;
	bool b_na = false;
	int ix = 0;
	while (c != '\n' && c != EOF)
	{
		int s = c - '1';
		c = getc(stdin);
		if (c == '\n' || c == EOF)
		{
			b_na = true;
			break;
		}
		int t = c - '1';
		oline[ix++] = letter[s][t];
		c = getc(stdin);
	}
	if (b_na)
		printf("NA\n");
	else
	{
		oline[ix] = NUL;
		printf("%s\n", oline);
	}
	if (c == EOF)
		return false;
	return true;
}
int
main(int argc, char **argv)
{
	while (true)
	{
		if (!ope_line())
			break;
	}
	return 0;
}
"""

python_code_snipet_1 = """
val = 5
size = 3
arr = [0] * size
i = 0
j = 1
total = 0
if val > 2:
    arr[j % size] = j
while i < size:
    arr[j] += 1
    score = arr[j+1]
    total += score * 2
    diff = total - score
    j = (j + 1) % size
    i += 1
last = arr[-1]
summary = diff + j
"""

c_prompt_dataflow_1 = """
**Question**: Is there information flow from `(s,18)` to `(oline,26)` in function `ope_line`? If so, provide a trace.

**Output**:
"""

# c_prompt_dataflow_1 = """
# **Question**: Is there information flow from `(s,18)` to `(oline,26)` in function `ope_line`?  
# If so, provide a precise trace of this flow **using only the line numbers shown in the annotated code**.   
# Then, **after the JSON output**, briefly explain the reason why the flow exists or does not exist.  

# **Output**:
# """

python_prompt_dataflow_1 = """
**Question**: Is there information flow from `(s,18)` to `(oline,26)` in function `ope_line`?  
If so, provide a precise trace of this flow **using only the line numbers shown in the annotated code**.   
Then, **after the JSON output**, briefly explain the reason why the flow exists or does not exist.  

**Output**:
"""

In [24]:
# 测试代码行分割
# print(c_with_li_num(c_code_snipet_1))

# Test-1：单轮次

question = c_prompt_1 + c_with_li_num(c_code_snipet_1) + c_prompt_dataflow_1

# question = c_with_li_num(c_code_snipet_1) + c_prompt_dataflow_1

# question = c_prompt_1 + c_with_li_num(python_code_snipet_1) + python_prompt_dataflow_1

print(llm_chat(question))

```json
{
  "InformationFlow": true,
  "Trace": [
    {
      "from": ["s", 18],
      "to":   ["oline", 26],
      "type": "data"
    }
  ]
}
```


In [ ]:
# 多轮次提问
def multi_round_qa(llm_chat, base_question, rounds=3):
    history = ""
    question = base_question
    for i in range(rounds):
        answer = llm_chat(history + question)
        print(f"---- Round {i+1} ----")
        print(answer)
        # 将当前回答加入“历史上下文”
        history += f"\n\n[Previous Answer Round {i+1}]:\n{answer}\n"
    return answer

print("multi_round_qa:\n")
multi_round_qa(llm_chat, question, 3)


multi_round_qa:

---- Round 1 ----
```json
{
  "InformationFlow": false
}
```
---- Round 2 ----
```json
{
  "InformationFlow": false
}
```
---- Round 3 ----
```json
{
  "InformationFlow": false
}
```
```json
{
  "InformationFlow": false
}
```
